In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math


In [5]:
def preprocess_text(text):
    text = text.lower()
    chars = sorted(list(set(text)))
    
    char_to_idx = {ch:i for i,ch in enumerate(chars)}
    idx_to_char = {i:ch for i,ch in enumerate(chars)}
    
    encoded = [char_to_idx[ch] for ch in text]
    
    return encoded, char_to_idx, idx_to_char, len(chars)


In [6]:
def create_sequences(encoded_text, seq_length):
    inputs = []
    targets = []
    
    for i in range(len(encoded_text) - seq_length):
        inputs.append(encoded_text[i:i+seq_length])
        targets.append(encoded_text[i+1:i+seq_length+1])  # next token for each position
        
    return inputs, targets


In [7]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)
    
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        
        self.num_heads = num_heads
        self.d_model = d_model
        self.head_dim = d_model // num_heads
        
        self.Wq = nn.Linear(d_model, d_model)
        self.Wk = nn.Linear(d_model, d_model)
        self.Wv = nn.Linear(d_model, d_model)
        
        self.fc_out = nn.Linear(d_model, d_model)
    
    def forward(self, x):
        B, T, C = x.shape
        
        Q = self.Wq(x)
        K = self.Wk(x)
        V = self.Wv(x)
        
        Q = Q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        # Causal mask
        mask = torch.tril(torch.ones(T, T))
        scores = scores.masked_fill(mask == 0, float("-inf"))
        
        attention = torch.softmax(scores, dim=-1)
        out = torch.matmul(attention, V)
        
        out = out.transpose(1, 2).contiguous().view(B, T, C)
        out = self.fc_out(out)
        
        return out


In [9]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )
        
        self.norm2 = nn.LayerNorm(d_model)
    
    def forward(self, x):
        attn_out = self.attention(x)
        x = self.norm1(x + attn_out)
        
        ff_out = self.ff(x)
        x = self.norm2(x + ff_out)
        
        return x


In [10]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, max_len):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len)
        
        self.block = TransformerBlock(d_model, num_heads, d_ff)
        
        self.fc_out = nn.Linear(d_model, vocab_size)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        
        x = self.block(x)
        
        logits = self.fc_out(x)
        return logits


In [11]:
text = """
artificial intelligence is transforming modern society.
it is used in healthcare finance education and transportation.
machine learning allows systems to improve automatically with experience.
data plays a critical role in training intelligent systems.
large datasets help models learn complex patterns.
deep learning uses multi layer neural networks.
neural networks are inspired by biological neurons.
each neuron processes input and produces an output.
training a neural network requires optimization techniques.
gradient descent minimizes the loss function.

natural language processing helps computers understand human language.
text generation is a key task in nlp.
language models predict the next word or character.
recurrent neural networks handle sequential data.
lstm and gru models address long term dependency problems.
however rnn based models are slow for long sequences.

transformer models changed the field of nlp.
they rely on self attention mechanisms.
attention allows the model to focus on relevant context.
transformers process data in parallel.
this makes training faster and more efficient.
modern language models are based on transformers.

education is being improved using artificial intelligence.
intelligent tutoring systems personalize learning.
automated grading saves time for teachers.
online education platforms use recommendation systems.
technology enhances the quality of learning experiences.

ethical considerations are important in artificial intelligence.
fairness transparency and accountability must be ensured.
ai systems should be designed responsibly.
data privacy and security are major concerns.
researchers continue to improve ai safety.

text generation models can create stories poems and articles.
they are used in chatbots virtual assistants and content creation.
generated text should be meaningful and coherent.
evaluation of text generation is challenging.
human judgement is often required.

continuous learning is essential in the field of ai.
research and innovation drive technological progress.
students should build strong foundations in mathematics.
programming skills are important for ai engineers.
practical experimentation enhances understanding.
"""


encoded, char_to_idx, idx_to_char, vocab_size = preprocess_text(text)

seq_length = 40
inputs, targets = create_sequences(encoded, seq_length)


In [12]:
model = TransformerModel(
    vocab_size=vocab_size,
    d_model=128,
    num_heads=4,
    d_ff=256,
    max_len=seq_length
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)


In [13]:
epochs = 15

for epoch in range(epochs):
    total_loss = 0
    
    for i in range(len(inputs)):
        x = torch.tensor([inputs[i]])
        y = torch.tensor([targets[i]])
        
        optimizer.zero_grad()
        
        output = model(x)
        loss = criterion(output.view(-1, vocab_size), y.view(-1))
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(inputs):.4f}")


Epoch 1, Loss: 2.1715
Epoch 2, Loss: 2.3934
Epoch 3, Loss: 2.4406
Epoch 4, Loss: 2.4125
Epoch 5, Loss: 2.3938
Epoch 6, Loss: 2.3423
Epoch 7, Loss: 2.2960
Epoch 8, Loss: 2.2508
Epoch 9, Loss: 2.2401
Epoch 10, Loss: 2.2250
Epoch 11, Loss: 2.2101
Epoch 12, Loss: 2.1926
Epoch 13, Loss: 2.1832
Epoch 14, Loss: 2.1986
Epoch 15, Loss: 2.1865


In [14]:
def generate_text(model, seed_text, seq_length, char_to_idx, idx_to_char, length=200):
    
    model.eval()
    generated = seed_text
    
    for _ in range(length):
        input_seq = generated[-seq_length:]
        input_indices = [char_to_idx[ch] for ch in input_seq]
        input_tensor = torch.tensor([input_indices])
        
        with torch.no_grad():
            output = model(input_tensor)
            last_token_logits = output[:, -1, :]
            predicted_idx = torch.argmax(last_token_logits, dim=-1).item()
        
        generated += idx_to_char[predicted_idx]
    
    return generated


In [17]:
seed = "artificial intelligence "
print(generate_text(model, seed, seq_length, char_to_idx, idx_to_char, 30))


artificial intelligence antin an an an an an an antent


In [18]:
print(sum(p.numel() for p in model.parameters()))

139933


In [19]:
from torchinfo import summary

dummy_input = torch.randint(0, vocab_size, (1, seq_length), dtype=torch.long)

summary(model, input_data=dummy_input)



Layer (type:depth-idx)                   Output Shape              Param #
TransformerModel                         [1, 40, 29]               --
├─Embedding: 1-1                         [1, 40, 128]              3,712
├─PositionalEncoding: 1-2                [1, 40, 128]              --
├─TransformerBlock: 1-3                  [1, 40, 128]              --
│    └─MultiHeadAttention: 2-1           [1, 40, 128]              --
│    │    └─Linear: 3-1                  [1, 40, 128]              16,512
│    │    └─Linear: 3-2                  [1, 40, 128]              16,512
│    │    └─Linear: 3-3                  [1, 40, 128]              16,512
│    │    └─Linear: 3-4                  [1, 40, 128]              16,512
│    └─LayerNorm: 2-2                    [1, 40, 128]              256
│    └─Sequential: 2-3                   [1, 40, 128]              --
│    │    └─Linear: 3-5                  [1, 40, 256]              33,024
│    │    └─ReLU: 3-6                    [1, 40, 256]        